# Project

In [9]:
using CSV, DataFrames, JuMP, Gurobi, Plots, LinearAlgebra

In [10]:
fires = CSV.read("data/wildfire_suppression.csv", DataFrame, header = true);
first(fires)

Row,report_id,fire_id,area,date,longitude,latitude,from_date,total_crews_sent
,Int64,Int64,Float64?,String31,Float64?,Float64?,String31,Float64
1,2714023,2714022,150.0,2015-05-06 10:15:00,-71.2519,43.7811,2015-05-05 09:30:00,45.0


In [97]:
# Turn strings into float
fires[!, :Column3] = string.(fires[:, :Column3])
fires[!, :Column5] = string.(fires[:, :Column5])
fires[!, :Column6] = string.(fires[:, :Column6])
fires[!, :Column8] = string.(fires[:, :Column8])
first(fires, 6)

Row,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
,String15,String7,Abstract…,String31,Abstract…,Abstract…,String31,String31
1,report_id,fire_id,area,date,longitude,latitude,date,total_crews_sent
2,2714023,2714022,150.0,2015-05-06 10:15:00,-71.2519444,43.7811111,2015-05-06 10:15:00,45.0
3,2714037,2714022,275.0,2015-05-08 00:30:00,-71.2519444,43.7811111,2015-05-08 00:30:00,45.0
4,2714050,2714022,275.0,2015-05-09 00:30:00,-71.2519444,43.7811111,2015-05-09 00:30:00,33.0
5,2714066,2714022,275.0,2015-05-10 13:00:00,-71.2519444,43.7811111,2015-05-10 13:00:00,0.0
6,2714082,2714081,205.0,2015-05-07 07:15:00,-67.0125,44.7916667,2015-05-07 07:15:00,21.0


In [99]:
fires[!, :Column3] = Float64.(fires[:, :Column3])

LoadError: MethodError: no method matching Float64(::String15)

[0mClosest candidates are:
[0m  (::Type{T})([91m::AbstractChar[39m) where T<:Union{AbstractChar, Number}
[0m[90m   @[39m [90mBase[39m [90m[4mchar.jl:50[24m[39m
[0m  (::Type{T})([91m::Base.TwicePrecision[39m) where T<:Number
[0m[90m   @[39m [90mBase[39m [90m[4mtwiceprecision.jl:266[24m[39m
[0m  (::Type{T})([91m::Complex[39m) where T<:Real
[0m[90m   @[39m [90mBase[39m [90m[4mcomplex.jl:44[24m[39m
[0m  ...


In [44]:
# Calculate Euclidian distance
function euclidean_distance(lat1, lon1, lat2, lon2)
    return sqrt((lat2 - lat1)^2 + (lon2 - lon1)^2)
end

# Calculate the Haversine distance
function haversine_distance(lat1, lon1, lat2, lon2)
    R = 6371.0 # Earth radius in kilometers
    dLat = deg2rad(lat2 - lat1)
    dLon = deg2rad(lon2 - lon1)
    lat1 = deg2rad(lat1)
    lat2 = deg2rad(lat2)

    a = sin(dLat/2)^2 + cos(lat1) * cos(lat2) * sin(dLon/2)^2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c
end

# Returns distance matrix of distances between fires
function get_distance_matrix(df, distance_function=1)
    n = nrow(df)
    dist_matrix = zeros(n, n)
    for i in 1:n
        for j in 1:n
            distance_matrix[i, j] = haversine_distance(df[i, :latitude], df[i, :longitude], df[j, :latitude], df[j, :longitude])
        end
    end
end;

distance_matrix = get_distance_matrix(fires)

LoadError: MethodError: no method matching -(::String15, ::String15)

[0mClosest candidates are:
[0m  -([91m::MutableArithmetics.Zero[39m, ::Any)
[0m[90m   @[39m [32mMutableArithmetics[39m [90m~/.julia/packages/MutableArithmetics/NIXlP/src/[39m[90m[4mrewrite.jl:63[24m[39m
[0m  -(::Any, [91m::MutableArithmetics.Zero[39m)
[0m[90m   @[39m [32mMutableArithmetics[39m [90m~/.julia/packages/MutableArithmetics/NIXlP/src/[39m[90m[4mrewrite.jl:64[24m[39m
